This notebook is incomplete but contains my work this far on creating a transformer. 

In [2]:
import numpy as np
import os
import pandas as pd
import pyodbc
import time
import scipy.stats as stats
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from collections import Counter
from datetime import datetime
import sys
import gc
from scipy.sparse import *
import pyarrow as pa
import torch
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.decomposition import PCA, IncrementalPCA, TruncatedSVD
import pickle 
import math

# Create Data for Transformer
### This model is looking at 90-day chunks of time, specifically between 90 days and 120 days pre-index date. This is NOT (yet) a visit-indexed model
Limit to data between 7 years (2555 days) and 90 days, inclusive prior to diagnosis. This also means that the psychosis_diagnosis date should be at least **90 days** before diagnosis

In [ ]:
num_days_prediction = 90
df_pop = pd.read_csv('population.csv')
df_pop.rename({'psychosis_dx_date':'psychosis_diagnosis_date'}, axis=1, inplace=True)
df_pop['psychosis_diagnosis_date'] = pd.to_datetime(df_pop['psychosis_diagnosis_date'], format="mixed")
df_pop['cohort_start_date'] = pd.to_datetime(df_pop['cohort_start_date'])
df_pop = df_pop.loc[(df_pop['cohort_start_date']-df_pop['psychosis_diagnosis_date']).dt.days >= num_days_prediction]

In [ ]:
all_visits = pd.read_csv('temporal_visits.csv')
all_conds = pd.read_csv('temporal_conditions.csv')
all_procedures = pd.read_csv('temporal_procedures.csv')
all_labs = pd.read_csv('temporal_labs.csv')
all_meds = pd.read_csv('temporal_medications.csv')

### Restrict data to timing that I want

In [ ]:
max_days = 2555

In [ ]:
all_meds = all_meds.loc[all_meds['person_id'].isin(df_pop['person_id'])]
all_meds['cohort_start_date'] = pd.to_datetime(all_meds['cohort_start_date'])
all_meds['drug_era_start_date'] = pd.to_datetime(all_meds['drug_era_start_date'])
all_meds['drug_era_end_date'] = pd.to_datetime(all_meds['drug_era_end_date'])
all_meds['censor_date'] = all_meds['cohort_start_date']-pd.Timedelta(num_days_prediction, 'd')

# remove any medications that start after the censor date
all_meds = all_meds.loc[all_meds['drug_era_start_date']<=all_meds['censor_date']]
print(min((all_meds['cohort_start_date']-all_meds['drug_era_start_date']).dt.days))
# replace end date of any medication that ends after the censor date with the censor date
all_meds.loc[all_meds['drug_era_end_date']>all_meds['censor_date'], 'drug_era_end_date'] = all_meds.loc[all_meds['drug_era_end_date']>all_meds['censor_date'], 'censor_date']
print(min((all_meds['cohort_start_date']-all_meds['drug_era_end_date']).dt.days))


# remove any medications that end before the max day
all_meds = all_meds.loc[(all_meds['cohort_start_date']-all_meds['drug_era_end_date']).dt.days <= max_days]
print(max((all_meds['cohort_start_date']-all_meds['drug_era_end_date']).dt.days))
#replace start date of any medication that starts before max days with max days
all_meds['early_cutoff'] = all_meds['cohort_start_date']-pd.Timedelta(max_days, 'd')
all_meds.loc[all_meds['drug_era_start_date']<all_meds['early_cutoff'], 'drug_era_start_date'] = all_meds.loc[all_meds['drug_era_start_date']<all_meds['early_cutoff'], 'early_cutoff']
print(max((all_meds['cohort_start_date']-all_meds['drug_era_start_date']).dt.days))

all_meds['days_to_cohort_start'] = (all_meds['cohort_start_date']-all_meds['drug_era_start_date']).dt.days

In [ ]:
all_visits = all_visits.loc[all_visits['person_id'].isin(df_pop['person_id'])]
all_visits['cohort_start_date'] = pd.to_datetime(all_visits['cohort_start_date'])
all_visits['visit_start_date'] = pd.to_datetime(all_visits['visit_start_date'])
all_visits['visit_end_date'] = pd.to_datetime(all_visits['visit_end_date'])
all_visits['censor_date'] = all_visits['cohort_start_date']-pd.Timedelta(num_days_prediction, 'd')

# remove any medications that start after the censor date
all_visits = all_visits.loc[all_visits['visit_start_date']<=all_visits['censor_date']]
print(min((all_visits['cohort_start_date']-all_visits['visit_start_date']).dt.days))
# replace end date of any medication that ends after the censor date with the censor date
all_visits.loc[all_visits['visit_end_date']>all_visits['censor_date'], 'visit_end_date'] = all_visits.loc[all_visits['visit_end_date']>all_visits['censor_date'], 'censor_date']
print(min((all_visits['cohort_start_date']-all_visits['visit_end_date']).dt.days))


# remove any medications that end before the max day
all_visits = all_visits.loc[(all_visits['cohort_start_date']-all_visits['visit_end_date']).dt.days <= max_days]
print(max((all_visits['cohort_start_date']-all_visits['visit_end_date']).dt.days))
#replace start date of any medication that starts before max days with max days
all_visits['early_cutoff'] = all_visits['cohort_start_date']-pd.Timedelta(max_days, 'd')
all_visits.loc[all_visits['visit_start_date']<all_visits['early_cutoff'], 'visit_start_date'] = all_visits.loc[all_visits['visit_start_date']<all_visits['early_cutoff'], 'early_cutoff']
print(max((all_visits['cohort_start_date']-all_visits['visit_start_date']).dt.days))

all_visits['days_to_cohort_start'] = (all_visits['cohort_start_date']-all_visits['visit_start_date']).dt.days

In [ ]:
all_conds = all_conds.loc[all_conds['person_id'].isin(df_pop['person_id'])]
all_conds['cohort_start_date'] = pd.to_datetime(all_conds['cohort_start_date'])
all_conds['condition_start_date'] = pd.to_datetime(all_conds['condition_start_date'])
all_conds['days_to_cohort_start'] = (all_conds['cohort_start_date']-all_conds['condition_start_date']).dt.days
all_conds = all_conds.loc[all_conds['days_to_cohort_start'] >= num_days_prediction]
all_conds = all_conds.loc[all_conds['days_to_cohort_start'] <= max_days]

print(all_conds['days_to_cohort_start'].min(),all_conds['days_to_cohort_start'].max())

In [ ]:
all_procedures = all_procedures.loc[all_procedures['person_id'].isin(df_pop['person_id'])]
all_procedures['cohort_start_date'] = pd.to_datetime(all_procedures['cohort_start_date'])
all_procedures['procedure_date'] = pd.to_datetime(all_procedures['procedure_date'])
all_procedures['days_to_cohort_start'] = (all_procedures['cohort_start_date']-all_procedures['procedure_date']).dt.days
all_procedures = all_procedures.loc[all_procedures['days_to_cohort_start'] >= num_days_prediction]
all_procedures = all_procedures.loc[all_procedures['days_to_cohort_start'] <= max_days]

print(all_procedures['days_to_cohort_start'].min(), all_procedures['days_to_cohort_start'].max())

In [ ]:
all_labs = all_labs.loc[all_labs['person_id'].isin(df_pop['person_id'])]
all_labs['cohort_start_date'] = pd.to_datetime(all_labs['cohort_start_date'])
all_labs['measurement_date'] = pd.to_datetime(all_labs['measurement_date'])
all_labs['days_to_cohort_start'] = (all_labs['cohort_start_date']-all_labs['measurement_date']).dt.days
all_labs = all_labs.loc[all_labs['days_to_cohort_start'] >= num_days_prediction]
all_labs = all_labs.loc[all_labs['days_to_cohort_start'] <= max_days]

print(all_labs['days_to_cohort_start'].min(), all_labs['days_to_cohort_start'].max())

### concatenate all dfs

In [ ]:
def drop_rare_occurrences(df, col_concept, col_id = 'person_id', size_pop = len(df_pop)):
    unique_occurrences = df[['person_id', col_concept]].drop_duplicates()
    unique_occurrences = unique_occurrences.value_counts(col_concept)
    common_occurrences = unique_occurrences[unique_occurrences/size_pop > 0.01].index
    return df.loc[df[col_concept].isin(common_occurrences)]
all_conds = drop_rare_occurrences(all_conds, 'condition_concept_id')
all_meds = drop_rare_occurrences(all_meds, 'drug_concept_id')
all_procedures = drop_rare_occurrences(all_procedures, 'procedure_concept_id')
all_labs = drop_rare_occurrences(all_labs, 'measurement_concept_id')

In [ ]:
all_labs = all_labs[['person_id', 'days_to_cohort_start', 'measurement_concept_id']].drop_duplicates()
all_labs.rename({'measurement_concept_id':'concept_id'}, axis=1, inplace=True)
len(all_labs)

In [ ]:
all_visits['los'] = (all_visits['visit_end_date']-all_visits['visit_start_date']).dt.days
los_df = all_visits.loc[all_visits['visit_concept_id'].isin([9201, 9203])]
los_df = los_df.loc[los_df['los']>0]
los_df = los_df[['person_id', 'visit_concept_id', 'los', 'days_to_cohort_start']]

list_temp_arrs = []
for i in tqdm(range(0,len(los_df))):
    n_repeats = los_df.iloc[i]['los']
    temp_arr = los_df.iloc[i].values.reshape(-1, 1).repeat(n_repeats, axis=0).reshape(4,n_repeats).T
    replace_days_to_cohort_start = np.arange(los_df.iloc[i]['days_to_cohort_start'], los_df.iloc[i]['days_to_cohort_start']-n_repeats, -1)
    temp_arr[:,-1] = replace_days_to_cohort_start
    list_temp_arrs.append(temp_arr)

los_df = pd.DataFrame(np.vstack(list_temp_arrs), columns = ['person_id', 'concept_id', 'los', 'days_to_cohort_start'])
los_df['concept_id'].replace({9201:9205, 9203:9207}, inplace=True)
los_df.drop(['los'], axis=1, inplace=True)
len(los_df)

In [ ]:
all_visits = all_visits[['person_id', 'days_to_cohort_start', 'visit_concept_id']].drop_duplicates()
all_visits.rename({'visit_concept_id':'concept_id'}, axis=1, inplace=True)
len(all_visits)

In [ ]:
all_meds = all_meds[['person_id', 'drug_exposure_count','drug_concept_id', 'days_to_cohort_start']].drop_duplicates()
# change the name to make merging the dfs easier later
all_meds.rename({'drug_concept_id':'concept_id'}, axis=1, inplace=True)

single_day_meds = all_meds.loc[all_meds['drug_exposure_count']==1]
print(len(single_day_meds))
multi_day_meds = all_meds.loc[all_meds['drug_exposure_count']>1]
print(len(multi_day_meds))
list_temp_arrs = []
for i in tqdm(range(0,len(multi_day_meds))):
    n_repeats = int(multi_day_meds.iloc[i]['drug_exposure_count'])
    temp_arr = multi_day_meds.iloc[i].values.reshape(-1, 1).repeat(n_repeats, axis=0).reshape(4,n_repeats).T
    replace_days_to_cohort_start = np.arange(multi_day_meds.iloc[i]['days_to_cohort_start'], multi_day_meds.iloc[i]['days_to_cohort_start']-n_repeats, -1)
    temp_arr[:,-1] = replace_days_to_cohort_start
    list_temp_arrs.append(temp_arr)
    
multi_day_meds = pd.DataFrame(np.vstack(list_temp_arrs), columns = single_day_meds.columns)
all_meds = pd.concat([multi_day_meds, single_day_meds])

all_meds.drop(['drug_exposure_count'], axis=1, inplace=True)

len(all_meds)

In [ ]:
all_conds = all_conds[['person_id', 'condition_concept_id', 'days_to_cohort_start']].drop_duplicates()
# change the name to make merging the dfs easier later
all_conds.rename({'condition_concept_id':'concept_id'}, axis=1, inplace=True)

In [ ]:
all_procedures = all_procedures[['person_id', 'days_to_cohort_start', 'procedure_concept_id']].drop_duplicates()
# change the name to make merging the dfs easier later
all_procedures.rename({'procedure_concept_id':'concept_id'}, axis=1, inplace=True)

In [ ]:
all_features = pd.concat([all_conds, all_meds, los_df, all_visits, all_labs, all_procedures])
all_features['days_to_cohort_start'] = all_features['days_to_cohort_start']//90
print(len(all_features))

### Create sparse dataframe

In [ ]:
grpr_row = all_features.groupby(['person_id', 'days_to_cohort_start']).grouper
idx_row = grpr_row.group_info[0]

grpr_col = all_features.groupby('concept_id').grouper
idx_col = grpr_col.group_info[0]

sparse_data = csr_matrix((all_features['concept_id'].values, (idx_row, idx_col)),shape=(grpr_row.ngroups, grpr_col.ngroups))

In [ ]:
df_index = grpr_row.result_index
df_columns = list(grpr_col.result_index)
sparse_df = pd.DataFrame.sparse.from_spmatrix(sparse_data, index = df_index, columns = df_columns)

In [ ]:
for i in sparse_df.columns:
    sparse_df[i] = sparse_df[i]/i

### Create multi-indexed dataframe that is a standard shape

In [ ]:
index_patients = sparse_df.index.get_level_values(0).unique()
patient_indices = np.arange(0,len(index_patients), 1)

cohort_start_days = np.arange(1, all_features['days_to_cohort_start'].max()+1, 1)
feature_indices = np.arange(0, sparse_df.shape[1], 1)

df_features = pd.DataFrame(data = 0, index = pd.MultiIndex.from_product([index_patients, cohort_start_days], names=["person_id", "months"]), columns = sparse_df.columns, dtype='int8')

In [ ]:
sparse_df = sparse_df[sparse_df.index.get_level_values(1)>0]
print(datetime.now())
df_features.loc[sparse_df.index] = sparse_df.values
print(datetime.now())

In [ ]:
# create my y_values vector in the correct order (according to index_patients)
mat_y = np.asarray(df_pop.set_index('person_id').loc[index_patients, 'sz_flag']).reshape(len(index_patients), 1)
df_pop = df_pop.loc[df_pop['person_id'].isin(index_patients)] # remove people who have no entries in person_id

### train test split and basic preprocessing

In [ ]:
# get first train-test split (to get test test data)
train_pop, test_pop, train_labels, test_labels, train_inds, test_inds = train_test_split(df_pop, mat_y, np.arange(0, len(df_pop), 1), random_state=23, test_size=0.2, stratify=mat_y)

In [ ]:
train_sparse_mat = csr_matrix(df_features.loc[train_pop['person_id']].values)
print(train_sparse_mat.shape)

test_sparse_mat = csr_matrix(df_features.loc[test_pop['person_id']].values)
print(test_sparse_mat.shape)

In [ ]:
test_sparse_mat = test_sparse_mat.todense()
train_sparse_mat = train_sparse_mat.todense()

In [ ]:
scaler = StandardScaler()
train_sparse_mat = scaler.fit_transform(train_sparse_mat)
print('done with fit/first transform')
test_sparse_mat = scaler.transform(test_sparse_mat)

In [ ]:
# create labels to go along with train/test mats
y_train = mat_y[train_inds]
print(len(y_train))
y_test = mat_y[test_inds]
print(len(y_test))

In [ ]:
del df_features
del sparse_df
gc.collect()

In [ ]:
np.savez('standard_scaled_mats.npz', train_sparse_mat, test_sparse_mat)
pd.DataFrame(y_train, index=train_inds).to_csv('training_labels.csv')
pd.DataFrame(y_test, index=test_inds).to_csv('testing_labels.csv')

In [ ]:
"""
pca = IncrementalPCA(n_components=1990, batch_size = 100)
chunk_size = 10000
for i in tqdm(range(0, len(train_sparse_mat)//chunk_size)):
    pca.partial_fit(train_sparse_mat[i*chunk_size : (i+1)*chunk_size,:])

print('done fitting')

pickle.dump(pca, open("psychosis_prediction/pca.pkl","wb"))
"""

## Read in data and transform to 3D

In [3]:
loaded_mats = np.load('standard_scaled_mats.npz')
y_train = pd.read_csv('training_labels.csv', index_col = 0)
y_test = pd.read_csv('testing_labels.csv', index_col = 0)

train_sparse_mat = loaded_mats['arr_0']
test_sparse_mat = loaded_mats['arr_1']
train_sparse_mat.shape

(2257164, 2026)

In [4]:
"""
pca = pickle.load(open("psychosis_prediction/pca.pkl",'rb'))
print(datetime.now())
train_sparse_mat = pca.transform(train_sparse_mat)
print(datetime.now())
test_sparse_mat = pca.transform(test_sparse_mat)
"""

'\npca = pickle.load(open("psychosis_prediction/pca.pkl",\'rb\'))\nprint(datetime.now())\ntrain_sparse_mat = pca.transform(train_sparse_mat)\nprint(datetime.now())\ntest_sparse_mat = pca.transform(test_sparse_mat)\n'

In [5]:
# validation split
train_pop, val_pop, train_labels, test_labels, train_inds, val_inds = train_test_split(y_train.index, y_train, np.arange(0, len(y_train), 1), random_state=24, test_size=0.1, stratify=y_train)
y_val = y_train.loc[val_pop]
y_train = y_train.loc[train_pop]

reshape gives me patients * timeseries * features
(https://stackoverflow.com/questions/54615882/how-to-convert-a-pandas-multiindex-dataframe-into-a-3d-array)

then I reorder to patients * features * timeseries (0,2,1)

In [6]:
num_train_pop = len(y_train)
num_test_pop = len(y_test)
num_val_pop = len(y_val)
len_sequence = 28
num_features = train_sparse_mat.shape[1]
#val_mat_features = df_val_features.astype('float32').values.reshape(num_val_pop, len_sequence, num_features).transpose(0, 2, 1)
test_mat_features = test_sparse_mat.reshape(num_test_pop, len_sequence, num_features).transpose(0, 2, 1)
print(test_mat_features.shape)
train_mat_features = train_sparse_mat.reshape(num_train_pop+num_val_pop, len_sequence, num_features).transpose(0, 2, 1)
print(train_mat_features.shape)

(20154, 2026, 28)
(80613, 2026, 28)


In [7]:
val_mat_features = train_mat_features[val_inds, :, :]
train_mat_features = train_mat_features[train_inds, :, :]

### Create Dataloader Objects

In [8]:
class StratifiedSampler(object):
    """Stratified Sampling

    https://github.com/ncullen93/torchsample/tree/master/torchsample
    """
    def __init__(self, class_vector, batch_size):
        """
        Arguments
        ---------
        class_vector : torch tensor
            a vector of class labels
        batch_size : integer
            batch_size
        """
        self.n_splits = int(class_vector.shape[0] / batch_size)
        self.class_vector = class_vector

    def gen_sample_array(self):
        try:
            from sklearn.model_selection import StratifiedShuffleSplit
        except:
            print('Need scikit-learn for this functionality')
        import numpy as np
        
        s = StratifiedShuffleSplit(n_splits=self.n_splits, test_size=0.5)
        X = torch.randn(self.class_vector.shape[0],2).numpy()
        y = self.class_vector
        s.get_n_splits(X, y)

        train_index, test_index = next(s.split(X, y))
        return np.hstack([train_index, test_index])

    def __iter__(self):
        return iter(self.gen_sample_array())

    def __len__(self):
        return len(self.class_vector)

In [9]:
train_sampler = StratifiedSampler(class_vector=np.asarray(y_train), batch_size=256)
train_data = torch.utils.data.TensorDataset(torch.Tensor(train_mat_features), torch.Tensor(y_train.values))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=256, sampler=train_sampler)

val_sampler = StratifiedSampler(class_vector=np.asarray(y_val), batch_size=256)
val_data = torch.utils.data.TensorDataset(torch.Tensor(val_mat_features), torch.Tensor(y_val.values))
val_loader = torch.utils.data.DataLoader(val_data, batch_size=256, sampler = val_sampler)

test_data = torch.utils.data.TensorDataset(torch.Tensor(test_mat_features), torch.Tensor(y_test.values))
test_loader = torch.utils.data.DataLoader(test_data, batch_size=256)

In [ ]:
torch.save(train_loader, 'train_loader.pth')
torch.save(val_loader, 'val_loader.pth')
torch.save(test_loader, 'test_loader.pth')

In [10]:
del train_sparse_mat
del train_data
del test_data
del val_data
del train_mat_features
del test_mat_features

gc.collect()

0

# Moving on to Model Training

In [15]:
class PositionalEncoding(nn.Module):
    # from pytorch 

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)


class EncoderModel(nn.Module):
    def __init__(self, input_size, n_features, hidden_size, num_layers, num_heads, output_size):
        super(EncoderModel, self).__init__()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.embedding = nn.Linear(n_features, hidden_size)
        #self.embedding = nn.Conv1d(n_features, hidden_size, 1, padding='same')
        self.pos_enc = PositionalEncoding(hidden_size, dropout=0.1)
        self.transformerenc = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, dim_feedforward=2048, dropout = 0.5),
                                                    num_layers = num_layers)
        
        self.regressor = nn.Sequential(
                           nn.Linear(hidden_size, hidden_size//2),
                           nn.Dropout(0.5),
                           nn.ReLU(),
                           nn.Linear(hidden_size//2, hidden_size//4),
                           nn.Dropout(0.5),
                           nn.ReLU(),
                           nn.Linear(hidden_size//4, hidden_size//8),
                           nn.Dropout(0.5),
                           nn.ReLU(),
                           nn.Linear(hidden_size//8, 1),
                           nn.Sigmoid())

    def forward(self, x):
        print(x.shape)
        x = x.permute(0, 2, 1) 
        print(x.shape)
        x = self.embedding(x) #linear looks at last dim so batch, sequence, features)
        # TransformerEncoder expects input in the shape (sequence, batch, features)
        print(x.shape)
        x = x.permute(1, 0, 2)
        print(x.shape)
        x = self.pos_enc(x)
        print(x.shape)
        x = self.transformerenc(x)
        print(x.shape)
        # Take the output from the last time step
        x = x[-1, :, :]
        print(x.shape)
        x = self.regressor(x)
        print(x.shape)
        return x


class TransformerModel(nn.Module):
    def __init__(self, input_size, n_features, hidden_size, num_layers, num_heads, output_size):
        
        """
        
        
        """
        super(TransformerModel, self).__init__()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.embedding = nn.Linear(n_features, hidden_size)
        self.transformer = nn.Transformer(d_model=hidden_size, nhead=num_heads, 
                                             num_encoder_layers=num_layers, num_decoder_layers=num_layers, 
                                             dim_feedforward=hidden_size*3, dropout=0.1, 
                                             activation='relu', device=self.device)
        
        self.fc1 = nn.Linear(hidden_size, 1)

    def forward(self, x, target):
        x = x.permute(0, 2, 1) 
        x = self.embedding(x) #linear looks at last dim so batch, sequence, features)
        # TransformerEncoder expects input in the shape (sequence, batch, features)
        #x = x.permute(2, 0, 1)
        x = x.permute(1, 0, 2)
        x = self.transformer(x)
        # Take the output from the last time step
        x = x[-1, :, :]
        x = self.sigmoid(x)
        return x


In [12]:
class Weighted_BCELoss(nn.Module):
    def __init__(self, weights):
        super(Weighted_BCELoss, self).__init__()
        self.weights = weights

    def forward(self, output, target, smooth=1):
        loss = self.weights[1] * (target * torch.log(output)) + self.weights[0] * ((1 - target) * torch.log(1 - output))
        
        return -torch.mean(loss)

In [13]:
device = torch.device("cuda:0") 
#loss_weights = torch.tensor(sum(y_train.values==0)/sum(y_train.values)).to(device)#; loss_criterion=torch.nn.BCEWithLogitsLoss(pos_weight = loss_weights)
#loss_weights = torch.tensor(np.asarray([sum(y_train.values)/len(y_train), 1-sum(y_train.values)/len(y_train)])).to(device)
loss_weights = torch.tensor(np.asarray([sum(y_train.values==0)/ sum(y_train.values==0), 0.25*sum(y_train.values==0)/sum(y_train.values)])).to(device)
def train(train_loader, model, device, optimizer, criterion=Weighted_BCELoss(weights=loss_weights)):
    model = model.to(device)
    model.train()
    epoch_loss = 0
    list_training_loss = []
    
    true_ys = []
    pred_ys = []
    for i, (signals, labels) in enumerate(train_loader):
        
        optimizer.zero_grad()
        signals, labels = signals.to(device), labels.to(device)
        
        outputs = model(signals)
        loss = criterion(outputs.squeeze(), labels.squeeze())
        epoch_loss += loss.item()
        loss.backward()
        list_training_loss.append(loss.item())
        optimizer.step()

        
        true_ys.append(labels.detach().cpu().numpy())
        pred_ys.append(outputs.detach().cpu().numpy())


    true_ys_flattened = np.concatenate(true_ys).ravel()
    pred_ys_flattened = np.concatenate(pred_ys).ravel()
    pred_labels = (pred_ys_flattened>0.5)*1
    
    auc_train = roc_auc_score(true_ys_flattened, pred_ys_flattened)
    f1_train = f1_score(true_ys_flattened, pred_labels)
    correct_label = accuracy_score(true_ys_flattened, pred_labels)

    return auc_train, f1_train, correct_label, np.mean(list_training_loss)

def test(test_loader, model, device, criteria=Weighted_BCELoss(weights=loss_weights), verbose=True):
    model.to(device)

    total_loss = 0
    list_testing_loss = []
    true_ys = []
    pred_ys = []
    for i, (x, y) in enumerate(test_loader):
        x, y = torch.Tensor(x.float()).to(device), torch.Tensor(y.float()).to(device)
        out = model(x)
        
        true_ys.append(y.detach().cpu().numpy())
        pred_ys.append(out.detach().cpu().numpy())

        loss = criteria(out.squeeze(), y.squeeze())
        total_loss += loss.item()
        list_testing_loss.append(loss.item())
    
    true_ys_flattened = np.concatenate(true_ys).ravel()
    pred_ys_flattened = np.concatenate(pred_ys).ravel()
    pred_labels = (pred_ys_flattened>0.5)*1
    
    print(sum(pred_labels))
    auc_test = roc_auc_score(true_ys_flattened, pred_ys_flattened)
    f1_test = f1_score(true_ys_flattened, pred_labels)
    correct_label = accuracy_score(true_ys_flattened, pred_labels)

    return auc_test, f1_test, correct_label, np.mean(list_testing_loss)


def train_model(model, train_loader, valid_loader, optimizer, n_epochs, device ,cv=0):
    train_loss_trend = []
    test_loss_trend = []

    for epoch in range(n_epochs + 1):
        auc_train, f1_train, accuracy_train, mean_loss_train = train(train_loader, model, device, optimizer)
        auc_test, f1_test, accuracy_test, mean_loss_test = test(valid_loader, model, device)
        
        train_loss_trend.append(mean_loss_train)
        test_loss_trend.append(mean_loss_test)
        #lr_scheduler.step()
        if epoch % 5 == 0:
            print('\nEpoch %d' % (epoch))
            print('Training ===>loss: ', mean_loss_train,
                  ' Accuracy: %.2f percent' % (100*accuracy_train),
                  ' AUC: %.2f' % (auc_train),
                 ' F1-Score: %.2f' % (f1_train))
            print('Test ===>loss: ', mean_loss_test,
                  ' Accuracy: %.2f percent' % (100*accuracy_test),
                  ' AUC: %.2f' % (auc_test),
                 ' F1-Score: %.2f' % (f1_test))
        """
        if epoch > 10:
            if (test_loss_trend[-2] < test_loss_trend[-1]) & (test_loss_trend[-3] < test_loss_trend[-1]):
                print('Breaking at epoch', epoch)
                break
        """

    # Save model and results
    """
    if not os.path.exists(os.path.join("./ckpt/", data)):
        os.mkdir(os.path.join("./ckpt/", data))
    if not os.path.exists(os.path.join("./plots/", data)):
        os.mkdir(os.path.join("./plots/", data))
    """
    #torch.save(model.state_dict(), 'psychosis_prediction/models/transformer1.pt')
    plt.plot(train_loss_trend, label='Train loss')
    plt.plot(test_loss_trend, label='Validation loss')
    plt.legend()
    

In [16]:
### Train the model
n_epochs = 20
input_size = val_mat_features.shape[2]
n_features = val_mat_features.shape[1]
hidden_size = 1024 # also try 512; 2048 did not work
num_layers = 4 #usually 4
num_heads = 8 # have tried 4, 8
output_size = 1
learning_rate = 1e-5
batch_size = 256

model = EncoderModel(input_size, n_features, hidden_size, num_layers, num_heads, output_size)
parameters = model.parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_model(model, train_loader, val_loader, optimizer, n_epochs, device,cv=0)
# last thing i did was add more layers

torch.Size([256, 2026, 28])
torch.Size([256, 28, 2026])
torch.Size([256, 28, 1024])
torch.Size([28, 256, 1024])
torch.Size([28, 256, 1024])
torch.Size([28, 256, 1024])
torch.Size([256, 1024])
torch.Size([256, 1])
torch.Size([256, 2026, 28])
torch.Size([256, 28, 2026])
torch.Size([256, 28, 1024])
torch.Size([28, 256, 1024])
torch.Size([28, 256, 1024])
torch.Size([28, 256, 1024])
torch.Size([256, 1024])
torch.Size([256, 1])
torch.Size([256, 2026, 28])
torch.Size([256, 28, 2026])
torch.Size([256, 28, 1024])
torch.Size([28, 256, 1024])
torch.Size([28, 256, 1024])
torch.Size([28, 256, 1024])
torch.Size([256, 1024])
torch.Size([256, 1])
torch.Size([256, 2026, 28])
torch.Size([256, 28, 2026])
torch.Size([256, 28, 1024])
torch.Size([28, 256, 1024])
torch.Size([28, 256, 1024])
torch.Size([28, 256, 1024])
torch.Size([256, 1024])
torch.Size([256, 1])
torch.Size([256, 2026, 28])
torch.Size([256, 28, 2026])
torch.Size([256, 28, 1024])
torch.Size([28, 256, 1024])
torch.Size([28, 256, 1024])
torch.Si

KeyboardInterrupt: 

In [ ]:
### Now look at summary stats for whole train data
list_outputs = []
list_labels = []
for _, [batch_x, batch_y] in enumerate(tqdm(val_loader)):
    outputs = model(batch_x.to(device))
    list_outputs.append(outputs)
    list_labels.append(batch_y)

list_outputs = np.round(torch.concat(list_outputs).to('cpu').detach().numpy())
list_labels = torch.concat(list_labels).to('cpu').numpy()

tn, fp, fn, tp = confusion_matrix(list_labels, list_outputs).ravel()
print('sensitivity', tp/(tp+fn))
print('specificity', tn/(tn+fp))
print('roc_auc', roc_auc_score(list_labels, list_outputs))
print('auprc', precision_score(list_labels, list_outputs))